#### geospatial2

 - geospatial exploratory file

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from rasterstats import zonal_stats
import richdem as rd
import elevation
import rasterio

In [ ]:
def get_dem(gdf):
    left, right, up, down = gdf.bounds
    elevation.clip(bounds=(left, right, up, down), output='/home/jovyan/work/data/utt-dem.tif')
    # clean up stale temporary files and fix the cache in the event of a server error
    elevation.clean()

def get_aspect():
    src = rd.LoadGDAL('/home/jovyan/work/data/utt-dem.tif')
    aspect = rd.TerrainAttribute(src, attrib='aspect')
    rd.SaveGDAL('/home/jovyan/work/data/utt-aspect.tif', aspect)
    
# def process_terrain_attributes():
#     elev = rasterio.open('/home/jovyan/work/data/utt-dem.tif')
#     !gdalwarp -t_srs EPSG:32644 /home/jovyan/work/data/utt.tif /home/jovyan/work/data/utt-dem-utm.tif
    
#     src = rd.LoadGDAL('/home/jovyan/work/data/utt-dem-utm.tif')
#     slope = rd.TerrainAttribute(src, attrib='slope_degrees')
#     rd.SaveGDAL('/home/jovyan/work/data/utt-slope-utm.tif', slope)
#     !gdalwarp -t_srs EPSG:4326 /home/jovyan/work/data/utt-slope-utm.tif /home/jovyan/work/data/utt-slope.tif

In [ ]:
#need to pick a road and add the data
#open the road files
road = gpd.read_file('/home/jovyan/work/notebooks/05-methods/closure-data/2021-06-28.gpkg')
road['elev_max'] = np.nan
road['elev_avg'] = np.nan
road['slope_max'] = np.nan
road['slope_avg'] = np.nan
road['aspect_avg'] = np.nan

In [ ]:
aspect

In [ ]:
for index, row in road.iterrows():
    geom = row['geometry']
    get_dem(geom)
    #process_terrain_attributes()
    get_aspect()
    elev = rasterio.open('/home/jovyan/work/data/utt-dem.tif')
    #slope = rasterio.open('/home/jovyan/work/data/utt-slope.tif')
    aspect = rasterio.open('/home/jovyan/work/data/utt-aspect.tif')
    elev_stats = zonal_stats(geom, 
            elev.read(1), affine=elev.transform,
            stats="count min mean max median")
#     slope_stats = zonal_stats(geom, 
#             slope.read(1), affine=slope.transform,
#             stats="count min mean max median")
    slope_stats = zonal_stats(geom, 
            slope2.read(1), affine=slope2.transform,
            stats="count min mean max median")
    aspect_stats = zonal_stats(geom, 
            aspect.read(1), affine=aspect.transform,
            stats="count mean")
    
    #update the road thing
    road.iloc[index, road.columns.get_loc("elev_max")] = elev_stats[0]['max']
    road.iloc[index, road.columns.get_loc("elev_avg")] = elev_stats[0]['mean']
    road.iloc[index, road.columns.get_loc("slope_max")] = slope_stats[0]['max']
    road.iloc[index, road.columns.get_loc("slope_avg")] = slope_stats[0]['mean']
    road.iloc[index, road.columns.get_loc("aspect_avg")] = aspect_stats[0]['mean']

In [ ]:
road.loc[road['status']==1]

In [ ]:
test = rasterio.open('output.tif')

In [ ]:
test.read(1)

In [ ]:
rda = rd.rdarray(test.read(1), no_data=-9999)

In [ ]:
#rd.no_data=-9999
rd.rdShow(rda, axes=False, cmap='magma', figsize=(8, 5.5))
plt.show()

In [ ]:
elev = rasterio.open('/home/jovyan/work/data/utt.tif')

#change projection
!gdalwarp -t_srs EPSG:32644 /home/jovyan/work/data/utt.tif /home/jovyan/work/data/output.tif
    
#slope
dem_path = '/home/jovyan/work/data/output.tif'
src = rd.LoadGDAL(dem_path)
slope = rd.TerrainAttribute(src, attrib='slope_degrees')
rd.rdShow(slope, axes=False, cmap='magma', figsize=(8, 5.5))
plt.show()

#curvature

#roughness

#aspect

#idk lol

In [ ]:
slope2 = rasterio.open('utt-slope.tif')

In [ ]:
geo = road.head(1)['geometry']

In [ ]:
geo

In [ ]:
slope_stats = zonal_stats(geo, 
            slope2.read(1), affine=slope2.transform,
            stats="count min mean max median")

In [ ]:
slope_stats

In [ ]:
for index, row in road.head(1).iterrows():
    geom = row['geometry']
    get_dem(geom)
    process_terrain_attributes()
    elev = rasterio.open('/home/jovyan/work/data/utt-dem.tif')
    slope = rasterio.open('/home/jovyan/work/data/utt-slope.tif')
    print(slope.crs)
    elev_stats = zonal_stats(geom, 
            elev.read(1), affine=elev.transform,
            stats="count min mean max median")
    slope_stats = zonal_stats(geom, 
            slope.read(1), affine=slope.transform,
            stats="count min mean max median")
    print(elev_stats)
    print(slope_stats)

In [ ]:
!gdalwarp -t_srs EPSG:32644 /home/jovyan/work/data/utt.tif /home/jovyan/work/data/output.tif

In [ ]:
#slope
#dem_path = "n30_e078_1arc_v3.tif"
dem = rd.LoadGDAL(dem_path)
slope = rd.TerrainAttribute(dem, attrib='slope_degrees')
rd.rdShow(slope, axes=False, cmap='magma', figsize=(8, 5.5))
plt.show()